In [4]:
# Jared Gibbs
# Truth in Taxation Inflation Rate Adjustment Modeling
# 8/1/2025

In [5]:
# Questions

# How do we deal with personal property rates since they are a 1-year lag of real property rates?
# How do we deal with new entities/ rates being created?

# inflate rate vs inflate revenue? 

# how to model the 10% growth cap? 

In [6]:
import pandas as pd

df_raw_inflation = pd.read_excel('Trimmed Mean PCE Inflation.xlsx')
df_raw_property = pd.read_excel('document (79).xls')

In [7]:
df_cleandata = df_raw_property[["Tax Year", "County Name", "Entity Name", "750 Tax_Rate_Real", "750 Tax_Rate_PP", "233b Total_Real", "233b Total_PP", "233b Total_CA"]]

df_cleandata = df_cleandata.merge(df_raw_inflation, how='left', on="Tax Year")

df_cleandata['inflation_rate_forwardlag'] = (df_cleandata['inflation_rate_forwardlag']/100) + 1

In [8]:
years = df_cleandata["Tax Year"].unique().tolist()

tax_base_types = ["233b Total_Real", "233b Total_PP", "233b Total_CA"]

entities = df_cleandata["Entity Name"].unique().tolist()


In [9]:
# Inflate tax rates
for year in years:
    for entity in entities:
        if year == 2015:
            alpha = df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity)]["750 Tax_Rate_Real"].sum() * df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity)]["inflation_rate_forwardlag"].sum()
            df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity),"inflated_real_rate"] = alpha
        
        if year >= 2016:
            alpha = df_cleandata.loc[(df_cleandata["Tax Year"] == year - 1) & (df_cleandata['Entity Name']== entity)]["inflated_real_rate"].sum() * df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity)]["inflation_rate_forwardlag"].sum()
            df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity),"inflated_real_rate"] = alpha


In [10]:
# Lag for Personal Property Rates
for year in years:
    for entity in entities:
        beta = df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity)]["inflated_real_rate"].sum()
        df_cleandata.loc[(df_cleandata["Tax Year"] == year - 1) & (df_cleandata['Entity Name']== entity),"inflated_pp_rate"] = beta



In [11]:
# Calculate Base Revenue
df_cleandata['base_real_revenue'] = df_cleandata['750 Tax_Rate_Real']* df_cleandata['233b Total_Real']
df_cleandata['base_pp_revenue'] = df_cleandata['750 Tax_Rate_PP']* df_cleandata['233b Total_PP']
df_cleandata['base_ca_revenue'] = df_cleandata['750 Tax_Rate_Real']* df_cleandata['233b Total_CA']
df_cleandata['base_total_revenue'] = df_cleandata['base_real_revenue'] + df_cleandata['base_pp_revenue'] + df_cleandata['base_ca_revenue'] 


In [ ]:
# Inflate Revenue 
# real property 
for year in years:
    for entity in entities:
        if year == 2015:
            gamma = df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity)]["base_real_revenue"].sum() * df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity)]["inflation_rate_forwardlag"].sum()
            df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity),"direct-inflated_real_revenue"] = gamma
        
        if year >= 2016:
            gamma = df_cleandata.loc[(df_cleandata["Tax Year"] == year - 1) & (df_cleandata['Entity Name']== entity)]["direct-inflated_real_revenue"].sum() * df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity)]["inflation_rate_forwardlag"].sum()
            df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity),"direct-inflated_real_revenue"] = gamma

# personal property 
for year in years:
    for entity in entities:
        if year == 2015:
            gamma = df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity)]["base_pp_revenue"].sum() * df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity)]["inflation_rate_forwardlag"].sum()
            df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity),"direct-inflated_pp_revenue"] = gamma
        
        if year >= 2016:
            gamma = df_cleandata.loc[(df_cleandata["Tax Year"] == year - 1) & (df_cleandata['Entity Name']== entity)]["direct-inflated_pp_revenue"].sum() * df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity)]["inflation_rate_forwardlag"].sum()
            df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity),"direct-inflated_pp_revenue"] = gamma


# Cenrally Assesed property 
for year in years:
    for entity in entities:
        if year == 2015:
            gamma = df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity)]["base_ca_revenue"].sum() * df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity)]["inflation_rate_forwardlag"].sum()
            df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity),"direct-inflated_ca_revenue"] = gamma
        
        if year >= 2016:
            gamma = df_cleandata.loc[(df_cleandata["Tax Year"] == year - 1) & (df_cleandata['Entity Name']== entity)]["direct-inflated_ca_revenue"].sum() * df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity)]["inflation_rate_forwardlag"].sum()
            df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity),"direct-inflated_ca_revenue"] = gamma

In [13]:
# Calculate Rate-Inflated Revenue
df_cleandata['rate-inflated_real_revenue'] = df_cleandata['inflated_real_rate']* df_cleandata['233b Total_Real']
df_cleandata['rate-inflated_pp_revenue'] = df_cleandata['inflated_pp_rate']* df_cleandata['233b Total_PP']
df_cleandata['rate-inflated_ca_revenue'] = df_cleandata['inflated_real_rate']* df_cleandata['233b Total_CA']
df_cleandata['rate-inflated_total_revenue'] = df_cleandata['rate-inflated_real_revenue'] + df_cleandata['rate-inflated_pp_revenue'] + df_cleandata['rate-inflated_ca_revenue']

# Compute rate-inflated differential
df_cleandata['rate-inflated_diff_real'] = df_cleandata['rate-inflated_real_revenue']- df_cleandata['base_real_revenue']
df_cleandata['rate-inflated_diff_pp'] = df_cleandata['rate-inflated_pp_revenue']- df_cleandata['base_pp_revenue']
df_cleandata['rate-inflated_diff_ca'] = df_cleandata['rate-inflated_ca_revenue']- df_cleandata['base_ca_revenue']
df_cleandata['rate-inflated_diff_total'] = df_cleandata['rate-inflated_total_revenue']- df_cleandata['base_total_revenue']




# Compute direct-inflated differential
df_cleandata['direct-inflated_diff_real'] = df_cleandata['direct-inflated_real_revenue']- df_cleandata['base_real_revenue']
df_cleandata['direct-inflated_diff_pp'] = df_cleandata['direct-inflated_pp_revenue']- df_cleandata['base_pp_revenue']
df_cleandata['direct-inflated_diff_ca'] = df_cleandata['direct-inflated_ca_revenue']- df_cleandata['base_ca_revenue']

df_cleandata['direct-inflated_total_revenue'] = df_cleandata['direct-inflated_diff_real'] + df_cleandata['direct-inflated_diff_pp'] + df_cleandata['direct-inflated_diff_ca']
df_cleandata['direct-inflated_diff_total'] = df_cleandata['direct-inflated_total_revenue']- df_cleandata['base_total_revenue']


In [14]:
# Graphing
